In [69]:
import pandas as pd
import numpy as np
from jinja2 import Template
from IPython.display import IFrame, HTML
import json

In [274]:
import os
os.chdir('D:/stock_data/big_companies')
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle Code</button>""", raw=True)

Toggle Code

In [5]:
apple = pd.read_csv('aapl.us.txt', sep=',', parse_dates=['Date'])
amazon = pd.read_csv('amzn.us.txt', sep=',', parse_dates=['Date'])
walmart = pd.read_csv('wmt.us.txt', sep=',', parse_dates=['Date'])
microsoft = pd.read_csv('msft.us.txt', sep=',', parse_dates=['Date'])
ibm = pd.read_csv('ibm.us.txt', sep=',', parse_dates=['Date'])
twitter = pd.read_csv('twtr.us.txt', sep=',', parse_dates=['Date']) 

In [13]:
twitter = twitter[twitter['Date'] >= '2014-01-01']
twitter.head()

,Date,Open,High,Low,Close,Volume,OpenInt
37,2014-01-02,65.00,67.50,64.40,67.50,29268962,0
38,2014-01-03,69.00,70.43,68.43,69.00,33235476,0
39,2014-01-06,64.99,66.87,63.50,66.29,27271097,0
40,2014-01-07,67.67,67.73,61.39,61.46,31781893,0
41,2014-01-08,58.71,61.26,57.92,59.29,27288675,0


In [14]:
ibm = ibm[ibm['Date'] >= '2014-01-01']
ibm.head()

,Date,Open,High,Low,Close,Volume,OpenInt
13085,2014-01-02,164.13,164.31,162.36,162.65,5186182,0
13086,2014-01-03,162.92,164.26,162.46,163.62,4635483,0
13087,2014-01-06,164.07,164.27,162.46,163.07,4640170,0
13088,2014-01-07,163.41,166.88,163.40,166.33,6766520,0
13089,2014-01-08,165.99,166.07,164.17,164.80,5251511,0


In [15]:
microsoft = microsoft[microsoft['Date'] >= '2014-01-01']
microsoft.head()

,Date,Open,High,Low,Close,Volume,OpenInt
7009,2014-01-02,33.863,33.909,33.636,33.690,33799452,0
7010,2014-01-03,33.726,33.746,33.184,33.463,34341184,0
7011,2014-01-06,33.410,33.446,32.738,32.756,48102902,0
7012,2014-01-07,32.939,33.083,32.827,33.010,39601380,0
7013,2014-01-08,32.640,32.765,32.257,32.419,66154830,0


In [16]:
walmart = walmart[walmart['Date'] >= '2014-01-01']
walmart.head()

,Date,Open,High,Low,Close,Volume,OpenInt
10469,2014-01-02,71.152,71.832,70.955,71.326,7645649,0
10470,2014-01-03,71.234,71.525,70.974,71.090,5484604,0
10471,2014-01-06,71.279,71.326,70.312,70.692,8056312,0
10472,2014-01-07,70.659,71.271,70.430,70.908,5546422,0
10473,2014-01-08,70.955,70.965,70.143,70.349,6607854,0


In [21]:
amazon = amazon[amazon['Date'] >= '2014-01-01']
amazon.head()

,Date,Open,High,Low,Close,Volume,OpenInt
4179,2014-01-02,398.80,399.36,394.02,397.97,2140046,0
4180,2014-01-03,398.29,402.71,396.22,396.44,2212891,0
4181,2014-01-06,395.85,397.00,388.42,393.63,3172058,0
4182,2014-01-07,395.04,398.47,394.29,398.03,1916210,0
4183,2014-01-08,398.47,403.00,396.04,401.92,2316680,0


In [23]:
apple = apple[apple['Date'] >= '2014-01-01']
apple.head()

,Date,Open,High,Low,Close,Volume,OpenInt
7390,2014-01-02,73.596,73.776,73.112,73.259,63381747,0
7391,2014-01-03,73.224,73.334,71.579,71.649,105949677,0
7392,2014-01-06,71.181,72.421,70.671,72.041,111371946,0
7393,2014-01-07,72.092,72.311,71.247,71.526,85837203,0
7394,2014-01-08,71.363,72.257,71.347,71.979,69742195,0


In [46]:
def df2c3(df):
    columns = []
    x_col=['x']
    x_col.extend(list(df['Date'].apply(lambda x: str(x)[:10]).values))
    columns.append(x_col)
    stock_list = ['Open', 'High', 'Low', 'Close']
    for col in stock_list:
        this_col = [col]
        this_col.extend(list(df[col].values))
        columns.append(this_col)
    return columns

In [140]:
for i in range(19):
    locals()["apple_{}".format(i)] = df2c3(apple.iloc[i*50: (i+1)*50])
apple_19 = df2c3(apple.iloc[950:])

In [271]:
head = """
        <!-- Load c3.css -->
        <link href='https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.10/c3.min.css' rel='stylesheet' type='text/css'/>
        <style>.c3-line { stroke-width: 1.5px; }</style>
        <style>.c3 svg{font: 15px sans-serif;}</style>
        <!-- Load d3.js and c3.js -->
        <script src='http://d3js.org/d3.v3.min.js' charset='utf-8'></script>
        <script src='http://cdnjs.cloudflare.com/ajax/libs/c3/0.4.18/c3.min.js'></script>
    """ 
template = """                                        
        <h4>Apple Stock</h4>
        <div id={{div_id}} style='width: 100%; height: 100%'></div>
        <button type='button' class='explore_distribution'>Explore Again</button>
        <script>
            var {{div_id}} = document.getElementById('{{div_id}}');
            function draw_all(){
            var chart = c3.generate({
                        bindto: '#{{div_id}}',
                        data: {
                            x: 'x',
                            columns: {{apple_0}}
                        },
                        axis: {
                            x: {
                                type: 'timeseries',
                                label: {text: 'Time', position: 'inner-center'},
                                tick: {
                                    format: '%y/%m/%d',
                                }
                            },
                            y: {
                            label: {text: 'Price', position: 'outer-center'}
                            }
                        }
                        }); 
                        
            setTimeout(function () {
            chart.flow({
            columns: {{apple_1}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_2}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_3}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_4}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_5}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_6}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_7}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_8}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_9}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_10}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_11}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_12}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_13}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_14}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_15}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_16}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_17}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_18}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_19}},
            duration: 3000,
            });
            }});}});}});}});}});}});}});}});}});}});}});}});}});}});}});}});}});},});}, 3000);
            }
          
            draw_all()

            
            d3.select('.explore_distribution').on('click', function(){
                draw_all();
            });
        </script>
                                    """
iframe = '<iframe srcdoc="{source}" src="" width="{w}" height="{h}" frameborder="0" sandbox="allow-scripts"></iframe>'

In [272]:
div_id="chart"
HTML(iframe.format(
            source=Template(head + template).render(
                                                    div_id=div_id.replace(" ", "_"),
                                                    apple_0 = apple_0,
            apple_1 = apple_1,
            apple_2 = apple_2,
            apple_3 = apple_3,
            apple_4 = apple_4,
            apple_5 = apple_5,
            apple_6 = apple_6,
            apple_7 = apple_7,
            apple_8 = apple_8,
            apple_9 = apple_9,
            apple_10 = apple_10,
            apple_11 = apple_11,
            apple_12 = apple_12,
            apple_13 = apple_13,
            apple_14 = apple_14,
            apple_15 = apple_15,
            apple_16 = apple_16,
            apple_17 = apple_17,
            apple_18 = apple_18,
            apple_19 = apple_19),
                                                    w=780,
                                                    h=450))

In [267]:
head = """
        <!-- Load c3.css -->
        <link href='https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.10/c3.min.css' rel='stylesheet' type='text/css'/>
        <style>.c3-line { stroke-width: 1.5px; }</style>
        <style>.c3 svg{font: 18px sans-serif;}</style>
        <style>#btn {
  background: #3498db;
  background-image: -webkit-linear-gradient(top, #3498db, #2980b9);
  background-image: -moz-linear-gradient(top, #3498db, #2980b9);
  background-image: -ms-linear-gradient(top, #3498db, #2980b9);
  background-image: -o-linear-gradient(top, #3498db, #2980b9);
  background-image: linear-gradient(to bottom, #3498db, #2980b9);
  -webkit-border-radius: 28;
  -moz-border-radius: 28;
  border-radius: 28px;
  font-family: Georgia;
  color: #ffffff;
  font-size: 25px;
  padding: 10px 20px 10px 20px;
  text-decoration: none;
}

#btn:hover {
  background: #3cb0fd;
  background-image: -webkit-linear-gradient(top, #3cb0fd, #3498db);
  background-image: -moz-linear-gradient(top, #3cb0fd, #3498db);
  background-image: -ms-linear-gradient(top, #3cb0fd, #3498db);
  background-image: -o-linear-gradient(top, #3cb0fd, #3498db);
  background-image: linear-gradient(to bottom, #3cb0fd, #3498db);
  text-decoration: none;
}
h1 { color: #111; font-family: 'Helvetica Neue', sans-serif; 
font-size: 60px; font-weight: bold; letter-spacing: -1px; line-height: 1;  }
</style>
        <!-- Load d3.js and c3.js -->
        <script src='http://d3js.org/d3.v3.min.js' charset='utf-8'></script>
        <script src='http://cdnjs.cloudflare.com/ajax/libs/c3/0.4.18/c3.min.js'></script>
    """ 
template = """                                        
        <h1>Apple Stock</h1>
        <div id={{div_id}} style='width: 1600px; height: 800px'></div>
        <button type='button' class='explore_distribution' id="btn" style='margin: 0 auto;'>Explore Again</button>
        <script>
            var {{div_id}} = document.getElementById('{{div_id}}');
            function draw_all(){
            var chart = c3.generate({
                        bindto: '#{{div_id}}',
                        data: {
                            x: 'x',
                            columns: {{apple_0}}
                        },
                        axis: {
                            x: {
                                type: 'timeseries',
                                label: {text: 'Time', position: 'outer-right'},
                                tick: {
                                    format: '%y/%m/%d',
                                }
                            },
                            y: {
                            label: {text: 'Price', position: 'outer-center'}
                            }
                        }
                        }); 
                        
            setTimeout(function () {
            chart.flow({
            columns: {{apple_1}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_2}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_3}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_4}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_5}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_6}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_7}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_8}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_9}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_10}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_11}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_12}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_13}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_14}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_15}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_16}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_17}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_18}},
            duration: 3000,
            done: function () {
            chart.flow({
            columns: {{apple_19}},
            duration: 3000,
            });
            }});}});}});}});}});}});}});}});}});}});}});}});}});}});}});}});}});},});}, 3000);
            }
          
            draw_all()

            
            d3.select('.explore_distribution').on('click', function(){
                draw_all();
            });
        </script>
                                    """
iframe = '<iframe srcdoc="{source}" src="" width="{w}" height="{h}" frameborder="0" sandbox="allow-scripts"></iframe>'
html = Template(head + template).render(
                                                    div_id=div_id.replace(" ", "_"),
                                                    apple_0 = apple_0,
            apple_1 = apple_1,
            apple_2 = apple_2,
            apple_3 = apple_3,
            apple_4 = apple_4,
            apple_5 = apple_5,
            apple_6 = apple_6,
            apple_7 = apple_7,
            apple_8 = apple_8,
            apple_9 = apple_9,
            apple_10 = apple_10,
            apple_11 = apple_11,
            apple_12 = apple_12,
            apple_13 = apple_13,
            apple_14 = apple_14,
            apple_15 = apple_15,
            apple_16 = apple_16,
            apple_17 = apple_17,
            apple_18 = apple_18,
            apple_19 = apple_19)
with open('chart_1.html', 'w') as f:
    f.write(html)